# ***File name: BDNet.py   (it contains defination of BDNet module)***

In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):              
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        return torch.cat([x, out], 1)

class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BottleneckBlock, self).__init__()
        inter_planes = out_planes * 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        out = self.conv2(self.relu(self.bn2(out)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return torch.cat([x, out], 1)

class TransitionBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(TransitionBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return F.avg_pool2d(out, 2)

class DenseBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, growth_rate, block, dropRate=0.0):
        super(DenseBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, growth_rate, nb_layers, dropRate)
    def _make_layer(self, block, in_planes, growth_rate, nb_layers, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(in_planes+i*growth_rate, growth_rate, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class DenseNet3(nn.Module):
    def __init__(self, depth, num_classes, growth_rate=12,
                 reduction=0.5, bottleneck=True, dropRate=0.0):
        super(DenseNet3, self).__init__()
        in_planes = 2 * growth_rate
        n = (depth - 4) / 3
        if bottleneck == True:
            n = n/2
            block = BottleneckBlock
        else:
            block = BasicBlock
        n = int(n)
        # 1st conv before any dense block
        self.conv1 = nn.Conv2d(3, in_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans1 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 2nd block
        self.block2 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans2 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 3rd block
        self.block3 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(in_planes, num_classes)
        self.in_planes = in_planes

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):    # for feed-forwarding
        out = self.conv1(x)
        out = self.trans1(self.block1(out))
        out = self.trans2(self.block2(out))
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.in_planes)
        return self.fc(out)

# **File name : execuation.ipynb **

In [0]:

# This function lodes the traning dataset and divide it into two non-overlapping validation and traning datasets 
import PIL.ImageOps
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler



def get_train_valid_loader(data_dir,
                           batch_size,
                           random_seed,
                           kfolde=10,
                           shuffle=True,
                           show_sample=False,
                           num_workers=1,
                           pin_memory=False,
                           shuffelthevaluditation=1):
    """
 
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - kfolde: the number of foldes in cross validation
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot  sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    -shuffelthevaluditation: the folding index of validation set
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """


     # load the dataset
     # define transforms transform_def for augmentaion or not as parameters
 
    train_dataset = CustomDatasetFromImagesForTraning(data_dir,transform_def=transforms.Compose([transforms.RandomRotation((-15,15), resample=False, expand=False, center=None),
             transforms.Scale(randint(32,35)),transforms.CenterCrop(32),transforms.ToTensor()]))
    valid_dataset =CustomDatasetFromImagesForTraning(data_dir,transform_def=transforms.Compose([
           
            transforms.ToTensor()
            ]))
    
    valid_size=(len(train_dataset)/kfolde)/(len(train_dataset))
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[:split* (int( shuffelthevaluditation) -1 )] + indices[split*int( shuffelthevaluditation):], indices[split * (int( shuffelthevaluditation) -1 ):split * int( shuffelthevaluditation)]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    ) 
    
    return (train_loader, valid_loader)
      

In [0]:
 # For custom datasets and this function is used to lode traning and Validation dataset

from torch.utils.data.dataset import Dataset 
import numpy as np
from PIL import Image
import PIL.ImageOps
import random
from random import randint
class CustomDatasetFromImagesForTraning(Dataset):
    def __init__(self, csv_path,transform_def):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform_def: pytorch transforms for transforms and tensor conversion or augmentation
        """
        # Transforms
        self.to_tensor =transform_def
        
        
        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Third column is for an operation indicator
        self.operation_arr = np.asarray(self.data_info.iloc[:, 2])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name,'r')
        img_as_img = PIL.ImageOps.invert(img_as_img)
        #convart the image into RGB format
        img_as_img=img_as_img.convert('RGB')
        new_width  = 32
        new_height = 32
        #resizeing the image into 32x32 
        img_as_img = img_as_img.resize((new_width, new_height), Image.ANTIALIAS)
        #adjust the contrast with factor 1.5
        img_as_img=transforms.functional.adjust_contrast(img_as_img,1.5)
        list11  = [1,0.2,0.3,0.5,0.6,0.7,0.8,0.9,1.3,1.5]
        #adjust the contrast
        img_as_img=transforms.functional.adjust_contrast(img_as_img,random.choice(list11))
       
        # Check if there is an operation
        some_operation = self.operation_arr[index]
        # If there is an operation
        if some_operation:
            # Do some operation on image
            # ...no operations is happen
            # ...
            pass
        # Transform image to tensor and augmentation purpose
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len
      
      

In [0]:
 # For custom datasets and this function is used to lode Testing  dataset

from torch.utils.data.dataset import Dataset 
import numpy as np
from PIL import Image
import PIL.ImageOps
from random import *
from random import randint
class CustomDatasetFromImagesForTesting(Dataset):
    def __init__(self, csv_path):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform_def: pytorch transforms for transforms and tensor conversion 
        """
        self.to_pure_tensor =transforms.Compose([
           
            transforms.ToTensor()
            ])
        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Third column is for an operation indicator
        self.operation_arr = np.asarray(self.data_info.iloc[:, 2])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name,'r')
        img_as_img = PIL.ImageOps.invert(img_as_img)
        #convart the image into RGB format
        img_as_img=img_as_img.convert('RGB')
        new_width  = 32
        new_height = 32
        #resizeing the image into 32x32 
        img_as_img = img_as_img.resize((new_width, new_height), Image.ANTIALIAS)
        #adjust the contrast with factor 1.5
        img_as_img=transforms.functional.adjust_contrast(img_as_img,1.5)
        # Check if there is an operation
        some_operation = self.operation_arr[index]
        # If there is an operation
        if some_operation:
            # Do some operation on image
            # ...no operations is happen
            # ...
            pass
        # Transform image to tensor
        img_as_tensor = self.to_pure_tensor(img_as_img)

        # Get label(class) of the image 
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len
      
      

In [0]:
import argparse
import os
import shutil
import time
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.onnx as ax
import BDNet as dn
if torch.cuda.is_available:
    torch.cuda.manual_seed_all(1)

from sklearn.model_selection import train_test_split    
torch.cuda.manual_seed_all(1)
torch.manual_seed(1)
# used for logging to TensorBoard
from tensorboard_logger import configure, log_value

import random


import os
import csv
from tqdm import tqdm
import torch,torchvision
import shutil
from PIL import Image
from torch.autograd import Variable
import easydict 



args = easydict.EasyDict({
        "batch_size": 32,
        "start_epoch":0,
        "test_batch_size": 64,
        "epochs":150,
        "lr":0.009,
        "momentum": .9,
        "seed":1,
        "print_freq":300,
        "weight_decay":1e-5,
        "layers":40,
        "growth":12,
        "droprate":0.09,

        "reduce":0.5,
        "bottleneck":True,

        "name":'BDNet',
        "kfold":10    
    })


"""
    "batch_size": Traning mini-batch size
    "start_epoch":Starting epoch number
    "test_batch_size": Testing mini-batch size
    "epochs":number of total epochs to run
    "lr":initial learning rate
    "momentum": momentum
    "seed":1,
    "print_freq":print frequency 
    "weight_decay":weight decay 
    "layers":total number of layers
    "growth":number of new channels per layer
    "droprate":dropout probability

    "reduce":compression rate in transition stage
    "bottleneck":To not use bottleneck block

    "name":model name
    "kfold":number of folding in cross vslidation   


"""

filename1="./bestcase5.pth"  # path to save model's best state 


    
best_prec1 = 0




def main():
    global args, best_prec1
    args = args
    
    kwargs = {'num_workers': 1, 'pin_memory': True}
    
    # set training and validation data loder
    train_loader, valid_loader =get_train_valid_loader(kfolde=args.kfold,shuffelthevaluditation=1,random_seed=False,data_dir='./ugbabsirbengprojdtaset/Nipu_bengli_digits_train_labels.csv',batch_size=args.batch_size, shuffle=True)
    print(len(train_loader))
    print(len(valid_loader)) 
    
    #set testing data loder
    cvs_file_fr_test_beng =CustomDatasetFromImagesForTesting('./ugbabsirbengprojdtaset/Nipu_bengli_digits_test_labels.csv')
    test_loader = torch.utils.data.DataLoader(dataset=cvs_file_fr_test_beng,batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    
    # create model
    model = dn.DenseNet3(args.layers, 10, args.growth, reduction=args.reduce,
                         bottleneck=args.bottleneck, dropRate=args.droprate)
    
    
    # get the number of model parameters
    print('Number of model parameters: {}'.format(
        sum([p.data.nelement() for p in model.parameters()])))
    
    # for training on multiple GPUs. 
    # Use CUDA_VISIBLE_DEVICES=0,1 to specify which GPUs to use
    # useing cuda
    model = model.cuda()
    cudnn.benchmark = True

    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                nesterov=True,
                                weight_decay=args.weight_decay)
    
    
    #oneing/creating required csv files for recording data
    
    
    with open('./testaccufor.csv','w') as f:

          thewriterfrtestaccu=csv.writer(f) 
          with open('./trainaccu.csv','w') as u:
              thewriterfrtrnaccu=csv.writer(u)
              with open('trainloss.csv','w') as fff:
                thewriterfrtrnloss=csv.writer(fff)
                with open('testloss.csv','w') as testloss:
                  thewriterfrtestloss=csv.writer(testloss)
                  with open('Validaccu.csv','w') as vaccu:
                    thewriterfrvaccu=csv.writer(vaccu)
            
                    for epoch in range(1, args.epochs):
                      adjust_learning_rate(optimizer, epoch)

                      # train for one epoch
                      lossrt,acuurate=train(train_loader, model, criterion, optimizer, epoch)
                      thewriterfrtrnaccu.writerow([str(epoch),float(acuurate)])
                      thewriterfrtrnloss.writerow([str(epoch),float(lossrt)])


                      # evaluate on validation set
                      loss11,prec11 = validate(valid_loader, model, criterion, epoch,'valid')
                      # evaluate on Testing set
                      loss1,prec1 = Testing(test_loader, model, criterion, epoch,'Test')
                      #print("validation accu: ",float(prec11),"testing accu: ",float(prec1))

                      # remember and save  best model state
                      is_best = prec1 > best_prec1
                      best_prec1 = max(prec1, best_prec1)
                      if(is_best):
                        torch.save(model, filename1)


                      thewriterfrtestaccu.writerow([str(epoch),float(prec1)])
                      thewriterfrvaccu.writerow([str(epoch),float(prec11)])
                      thewriterfrtestloss.writerow([str(epoch),float(loss1)])
                      train_loader, valid_loader =get_train_valid_loader(kfolde=args.kfold,shuffelthevaluditation=(epoch%args.kfold)+1,random_seed=False,data_dir='./ugbabsirbengprojdtaset/Nipu_bengli_digits_train_labels.csv',batch_size=args.batch_size, shuffle=True)
                      print("validation accu: ",float(prec11),"testing accu: ",float(prec1),'till now best accuracy: ', best_prec1)
                      



              
              
              
              
              
              

    #printing the best result 
    
    print('Best accuracy: ', best_prec1)
    
    

def train(train_loader, model, criterion, optimizer, epoch):
    """Train for one epoch on the training set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        target = target.cuda(async=True)
        input = input.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        
        losses.update(loss.data, input.size(0))
        top1.update(prec1, input.size(0))

        # compute gradient and do SGD step     --backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      loss=losses, top1=top1))

    #print(losses.avg , top1.avg)
    return (losses.avg , top1.avg)

def validate(val_loader, model, criterion, epoch,mode):
    """Perform validation on the validation set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
            
        
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data, input.size(0))
        top1.update(prec1, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
          
         
          print(mode,'Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      top1=top1))
            

    return(losses.avg,top1.avg)


def Testing(val_loader, model, criterion, epoch,mode):
    """Perform Testing on the Testing set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
            
        
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data, input.size(0))
        top1.update(prec1, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
          
         
          print(mode,'Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      top1=top1))
            

    return(losses.avg,top1.avg)


  
  


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    if (epoch>80):
      for param_group in optimizer.param_groups:
        param_group['lr']=args.lr * .15
    

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

if __name__ == '__main__':
    main()

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


546
61
Number of model parameters: 176122
Epoch: [1][0/546]	Time 1.038 (1.038)	Loss 2.3446 (2.3446)	Prec@1 9.375 (9.375)
Epoch: [1][300/546]	Time 0.070 (0.075)	Loss 1.7779 (2.0858)	Prec@1 37.500 (25.031)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:250: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:251: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


valid Test: [0/61]	Time 0.124 (0.124)	Loss 1.5654 (1.5654)	Prec@1 43.750 (43.750)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:294: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:295: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test Test: [0/63]	Time 0.468 (0.468)	Loss 1.4132 (1.4132)	Prec@1 37.500 (37.500)
validation accu:  36.97782516479492 testing accu:  38.92500305175781 till now best accuracy:  tensor(38.9250, device='cuda:0')
Epoch: [2][0/546]	Time 0.197 (0.197)	Loss 1.1827 (1.1827)	Prec@1 62.500 (62.500)
Epoch: [2][300/546]	Time 0.071 (0.072)	Loss 0.8736 (1.0030)	Prec@1 65.625 (69.871)
valid Test: [0/61]	Time 0.113 (0.113)	Loss 0.6474 (0.6474)	Prec@1 75.000 (75.000)
Test Test: [0/63]	Time 0.150 (0.150)	Loss 0.5332 (0.5332)	Prec@1 81.250 (81.250)
validation accu:  82.4651870727539 testing accu:  84.12500762939453 till now best accuracy:  tensor(84.1250, device='cuda:0')
Epoch: [3][0/546]	Time 0.195 (0.195)	Loss 0.4941 (0.4941)	Prec@1 81.250 (81.250)
Epoch: [3][300/546]	Time 0.074 (0.073)	Loss 0.3418 (0.4869)	Prec@1 90.625 (84.292)
valid Test: [0/61]	Time 0.115 (0.115)	Loss 0.2144 (0.2144)	Prec@1 96.875 (96.875)
Test Test: [0/63]	Time 0.144 (0.144)	Loss 0.2437 (0.2437)	Prec@1 92.188 (92.188)
validation a

In [6]:
import argparse
import os
import shutil
import time
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.onnx as ax
import BDNet as dn
if torch.cuda.is_available:
    torch.cuda.manual_seed_all(1)

from sklearn.model_selection import train_test_split    
torch.cuda.manual_seed_all(1)
torch.manual_seed(1)
# used for logging to TensorBoard
from tensorboard_logger import configure, log_value

import random


import os
import csv
from tqdm import tqdm
import torch,torchvision
import shutil
from PIL import Image
from torch.autograd import Variable
import easydict 


# lode the Best model's state , that is parfect fitted model from above traning process
the_model = torch.load('./bestcase5.pth')  
#create an array of 10x10 dimension and initialize each field with 0
c_mat= [[0 for i in range(10)] for i in range(10)]
for kkk in range(10):
  for hhh in range(10):
    c_mat[kkk][hhh]=0
#lode the IsI test datasamples
cvs_file_fr_test_beng =CustomDatasetFromImagesForTesting('./ugbabsirbengprojdtaset/Nipu_bengli_digits_test_labels.csv')

tesst_loader = torch.utils.data.DataLoader(dataset=cvs_file_fr_test_beng,batch_size=1, shuffle=True)
ttl=0
crect=0
for i, (input, target) in enumerate(tesst_loader):
  input = input.cuda()
  input_var = torch.autograd.Variable(input, volatile=True)
  output = the_model(input_var)
  res=output.max(1, keepdim=True)[1]
  # fff holdes the predicted output class by our model
  # target holdes the actual output class 
  fff=res.tolist()[0][0]
  if(int(target)==int(fff)) :     #if the model predict correctly
     crect = crect+1
  ttl=ttl+1
  c_mat[int(target)][int(fff)]=c_mat[int(target)][int(fff)] + 1
print("\n\n Confution matrics on ISI Test DataSamples:-\n")
for kkk in range(10):
  for hhh in range(10):
    print(c_mat[kkk][hhh],end="\t")
  print("\n")   
print("Final testing accuracy: ",(crect/ttl)*100,"%")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.




 Confution matrics on ISI Test DataSamples:-

398	0	0	2	0	0	0	0	0	0	

0	399	0	0	0	0	0	0	0	1	

0	0	400	0	0	0	0	0	0	0	

0	0	0	400	0	0	0	0	0	0	

0	1	0	0	399	0	0	0	0	0	

0	0	0	0	0	399	1	0	0	0	

0	0	0	0	0	1	399	0	0	0	

0	0	0	0	1	0	0	399	0	0	

0	0	0	0	0	0	0	0	400	0	

0	2	0	0	0	1	0	0	0	397	

Final testing accuracy:  99.75 %


In [7]:
#create an array of 10x10 dimension and initialize each field with 0
c_mat= [[0 for i in range(10)] for i in range(10)]
for kkk in range(10):
  for hhh in range(10):
    c_mat[kkk][hhh]=0

#loadeing our own dataset     
    
cvs_file_fr_test_beng =CustomDatasetFromImagesForTesting('./own/owndta.csv')

tesst_loader = torch.utils.data.DataLoader(dataset=cvs_file_fr_test_beng,batch_size=1, shuffle=True)
ttl=0
crect=0
for i, (input, target) in enumerate(tesst_loader):
  input = input.cuda()
  input_var = torch.autograd.Variable(input, volatile=True)
  output = the_model(input_var)
  res=output.max(1, keepdim=True)[1]
  # fff holdes the predicted output class by our model
  # target holdes the actual output class 
  fff=res.tolist()[0][0]
  if(int(target)==int(fff)) :     #if the model predict correctly
     crect = crect+1
  ttl=ttl+1
  c_mat[int(target)][int(fff)]=c_mat[int(target)][int(fff)] + 1
  
print("\n\n Confution matrics on Custome made Our own DataSamples:-\n")
for kkk in range(10):
  for hhh in range(10):
    print(c_mat[kkk][hhh],end="\t")
  print("\n")   
print("Final testing accuracy: ",(crect/ttl)*100," %")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app




 Confution matrics on Custome made Our own DataSamples:-

99	1	0	0	0	0	0	0	0	0	

0	100	0	0	0	0	0	0	0	0	

0	0	100	0	0	0	0	0	0	0	

0	0	0	100	0	0	0	0	0	0	

0	0	0	0	100	0	0	0	0	0	

1	0	0	0	0	96	3	0	0	0	

0	0	0	0	0	0	99	0	1	0	

0	0	0	1	0	0	0	99	0	0	

0	0	1	0	0	0	0	0	99	0	

0	0	0	0	0	0	0	0	0	100	

Final testing accuracy:  99.2  %
